# Boundary layer hyperbolic geometry with Koiter shell (linear, HHJ)
We use a Zienkiewicz-Zhu (ZZ) error estimator to identify singularities and boundary layers and generate a sequence of grids. 

In [ ]:
from ngsolve import *
from netgen.csg import *
from netgen.meshing import MeshingStep
from ngsolve.webgui import Draw
import netgen.meshing as meshing

order = 6

L = 1
thickness = 0.00001
print("layer thickness =", thickness**(1/4))
rho = 360

E = 2e10
nu = 0.3

#gravity g=1
force    = 1*rho*thickness*CF( (0,0,-0.01) )

clamped  = "left"
free     = "right|bottom"
symmetry = "top"

mapping = lambda x,y,z: (x-0.5,(y/2-0.5),(x-0.5)**2 - (y/2-0.5)**2)
geom = meshing.SurfaceGeometry(mapping)
mesh = Mesh(geom.GenerateMesh(quads=False, nx=6, ny=6)).Curve(order)
Draw(mesh)

In [ ]:
interpolateMembrane=True
def MaterialStress(mat):
    return E/(1-nu**2)*((1-nu)*mat+nu*Trace(mat)*Id(3))

def MaterialStressInv(mat):
    return (1+nu)/E*(mat-nu/(nu+1)*Trace(mat)*Id(3))

t = thickness

def Solve(mesh,order):
    fesU = VectorH1(mesh, order=order, dirichletx_bbnd="left", dirichlety_bbnd="left|top", dirichletz_bbnd="left")
    fesS = HDivDivSurface(mesh, order=order-1, discontinuous=True)
    fesH = NormalFacetSurface(mesh, order=order-1, dirichlet_bbnd="left|top")

    fes = fesU*fesS*fesH
    (u,sigma,uh), (du,dsigma,duh) = fes.TnT()
    sigma,dsigma,uh,duh = sigma.Trace(),dsigma.Trace(),uh.Trace(),duh.Trace()

    fesRR = HCurlCurl(mesh, order=order-1)

    n       = specialcf.normal(3)
    tangent = specialcf.tangential(3)
    nel     = Cross(n,tangent)

    Ptau = Id(3) - OuterProduct(n,n)
    gradu = Grad(u).Trace()
    graddu = Grad(du).Trace()

    solution = GridFunction(fes)

    Hn = CF( (u.Operator("hesseboundary").trans*n), dims=(3,3) )
    dHn = CF( (du.Operator("hesseboundary").trans*n), dims=(3,3) )

    a = BilinearForm(fes, symmetric=True, condense=True)
    # membrane part
    if interpolateMembrane:
        a += (t*InnerProduct(MaterialStress(Interpolate(Sym(Ptau*gradu),fesRR)), Interpolate(Sym(Ptau*graddu),fesRR)))*ds
    else:
        a += (t*InnerProduct(MaterialStress(Sym(Ptau*gradu)), Sym(Ptau*graddu)))*ds
    #bending part
    a += (-12/t**3*InnerProduct(MaterialStressInv(sigma),dsigma))*ds
    a += (InnerProduct(dsigma,Hn) + InnerProduct(sigma,dHn))*ds
    a += ( -(sigma*nel*nel)*(graddu.trans*n*nel) - (dsigma*nel*nel)*(gradu.trans*n*nel) + (uh*nel)*(dsigma*nel)*nel + (duh*nel)*(sigma*nel)*nel )*ds(element_boundary=True)


    f = LinearForm(fes)
    f += force*du*ds
    
    a.Assemble()
    f.Assemble()
    
    r = f.vec.CreateVector()
    if a.condense:     
        r.data = f.vec
        r.data += a.harmonic_extension_trans * r
        inv = a.mat.Inverse(fes.FreeDofs(True), inverse="sparsecholesky")
        solution.vec.data = inv*r
        solution.vec.data += a.harmonic_extension * solution.vec
        solution.vec.data += a.inner_solve * r
    else:
        inv = a.mat.Inverse(self.fes.FreeDofs(), inverse="umfpack")
        r.data = f.vec
        solution.vec.data = inv*r
            
    gfu,gfsigma,gfuh = solution.components
    Draw(gfu,mesh,deformation=True)
    return solution

In [ ]:
fes_H = VectorH1(mesh,order=order, autoupdate=True)

gf_mem1 = GridFunction(fes_H, autoupdate=True)
gf_mem2 = GridFunction(fes_H, autoupdate=True)
gf_mem3 = GridFunction(fes_H, autoupdate=True)
gf_mem = CF( (gf_mem1, gf_mem2, gf_mem3), dims=(3,3) )

gf_bend1 = GridFunction(fes_H, autoupdate=True)
gf_bend2 = GridFunction(fes_H, autoupdate=True)
gf_bend3 = GridFunction(fes_H, autoupdate=True)
gf_bend = CF( (gf_bend1, gf_bend2, gf_bend3), dims=(3,3) )


fesl2 = SurfaceL2(mesh,order=0,autoupdate=True)
gferr_mem   = GridFunction(fesl2, autoupdate=True)
gferr_bend  = GridFunction(fesl2, autoupdate=True)
gferr       = GridFunction(fesl2, autoupdate=True)

def EstimateAndRefine(solution,mesh):
    Ptau = Id(3) - OuterProduct(specialcf.normal(3),specialcf.normal(3))
    if interpolateMembrane:
        membrane_strain = Interpolate(Sym(Ptau*Grad(solution.components[0])),HCurlCurl(mesh, order=order-1))
    else:
        membrane_strain = Sym(Ptau*Grad(solution.components[0]))
    cf_mem   = thickness*MaterialStress(membrane_strain)
    cf_bend  = solution.components[1]
    
    gf_mem1.Set(CF( (cf_mem[0,0],cf_mem[0,1],cf_mem[0,2]) ), definedon=mesh.Boundaries(".*"))
    gf_mem2.Set(CF( (cf_mem[1,0],cf_mem[1,1],cf_mem[1,2]) ), definedon=mesh.Boundaries(".*"))
    gf_mem3.Set(CF( (cf_mem[2,0],cf_mem[2,1],cf_mem[2,2]) ), definedon=mesh.Boundaries(".*"))
    
    gf_bend1.Set(CF( (cf_bend[0,0],cf_bend[0,1],cf_bend[0,2]) ), definedon=mesh.Boundaries(".*"))
    gf_bend2.Set(CF( (cf_bend[1,0],cf_bend[1,1],cf_bend[1,2]) ), definedon=mesh.Boundaries(".*"))
    gf_bend3.Set(CF( (cf_bend[2,0],cf_bend[2,1],cf_bend[2,2]) ), definedon=mesh.Boundaries(".*"))
        
    
    err_cf_mem   = InnerProduct(1/thickness*MaterialStressInv(gf_mem-cf_mem),gf_mem-cf_mem)
    err_cf_bend  = InnerProduct(12/thickness**3*MaterialStressInv(gf_bend-cf_bend),gf_bend-cf_bend)
    
    err_mem = Integrate(err_cf_mem, mesh, BND, element_wise=True,order=max(5,2*order))
    for el in mesh.Elements(BND):
        gferr_mem.vec[el.nr] = err_mem[el.nr]
        gferr.vec[el.nr] = err_mem[el.nr]
            
    err_bend = Integrate(err_cf_bend, mesh, BND, element_wise=True,order=max(5,2*order))

    for el in mesh.Elements(BND):
        gferr_bend.vec[el.nr] = err_bend[el.nr]
        gferr.vec[el.nr] += err_bend[el.nr]
            
    maxerr = max(gferr.vec)
    
    for el in mesh.Elements(BND):    
        mesh.SetRefinementFlag(el, gferr.vec[el.nr] > 0.25*maxerr)
    mesh.Refine(True)
    mesh.Curve(order)
    return

In [ ]:
with TaskManager():
    for i in range(13):
        solution=Solve(mesh,order)
        EstimateAndRefine(solution,mesh)